In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
import pylab as py
import random
import pandas as pd
from matplotlib import gridspec
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from datetime import datetime
import time
import corner
import emcee
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy import odr
from ltsfit.lts_linefit import lts_linefit

In [2]:
####################################### Set Axes
def set_axes(ax, xlim=None, ylim=None, fontsize=16, twinx=True, twiny=True, minor=True, inout='in'):
        
        if not ylim is None:
            ax.set_ylim(ylim)
        else:
            ylim = ax.get_ylim() 
            
        if not xlim is None:    
            ax.set_xlim(xlim) 
        else:
            xlim = ax.get_xlim()
            
        ax.tick_params(which='major', length=6, width=1., direction=inout)
#         if minor:
        ax.tick_params(which='minor', length=3, color='#000033', width=1.0, direction=inout)  
        
        if twiny:
            y_ax = ax.twinx()
            y_ax.set_ylim(ylim)
            y_ax.set_yticklabels([])
            y_ax.minorticks_on()
            y_ax.tick_params(which='major', length=6, width=1., direction=inout)
            if minor:
                y_ax.tick_params(which='minor', length=3, color='#000033', width=1.0, direction=inout) 
        
        if twinx:
            x_ax = ax.twiny()
            x_ax.set_xlim(xlim)
            x_ax.set_xticklabels([])
            x_ax.minorticks_on()
            x_ax.tick_params(which='major', length=6, width=1.0, direction=inout)
            if minor:
                x_ax.tick_params(which='minor', length=3, color='#000033', width=1.0, direction=inout)     

        for tick in ax.xaxis.get_major_ticks():
                    tick.label.set_fontsize(fontsize) 
        for tick in ax.yaxis.get_major_ticks():
                    tick.label.set_fontsize(fontsize) 
        
        return x_ax, y_ax

In [3]:
def getDM(D, mag, gi, Wlog, F21, alfa, beta, slope, zp, V3k, mc):
    
    Mb = 10**(slope*(Wlog-2.5)+zp)
    
    M21 = 2.356E5 * F21 * (D**2)
    M21 = 10**(np.log10(M21) + mc*V3k)
    
    logX = beta + alfa*gi   
    Mag  = -2.5*(np.log10(Mb - 1.33*M21) - logX)
       
    DM = mag - Mag
    D = 10**((DM-25)/5)
    
    return D


def interpol(X, Y):
    
    X0 = X[~np.isnan(Y)]
    Y0 = Y[~np.isnan(Y)]
      
    N = len(X0)
    
    for i in range(N-1):
        
        if Y0[i]<0 and Y0[i+1]>0:
            break

    return -Y0[i]*(X0[i+1]-X0[i])/(Y0[i+1]-Y0[i])+X0[i]

In [4]:
t1 =  datetime.now()
print(t1)


# model parameters
alfa0 = 0.510
beta0 = 1.619
slope0 = 3.421
zp0 = 10.41065
mc = -5.967700322195351e-05

# galaxy parameters pgc = 4 -> DM=33.41
mag = 15.12
gi = 15.59 - 15.12
Wlog = 2.189
F21 = 1.91
V3k = 4706

# galaxy parameters pgc = 70  -> DM=35.36
mag = 13.24         # -> 0.05
gi = 13.87 - 13.24  # -> 0.05*sqrt(2.)
Wlog = 2.636        # -> Wlog_e
F21 = 8.33          # -> F21e
V3k = 7040          

npzName = "MLratio_i_gi.npz"
npzfile = np.load(npzName)
samples_ = npzfile['arr_0']
samples_ = samples_[np.where(samples_[:,0]>0.48)]
samples_ = samples_[(samples_[:,1]>1.56)&(samples_[:,1]<1.68)]
samples_ = samples_[np.where(samples_[:,3]>10.31)]

alfa = samples_[:,0]
beta = samples_[:,1]
slope = samples_[:,2]
zp = samples_[:,3]-10.363+zp0

Nsimul = 1000
DM = np.zeros(Nsimul) + np.nan
n, m = samples_.shape

for i in range(Nsimul):
    
    try:
    
        j = np.random.randint(n)
        D = np.arange(0.5,800,0.5)
        delta = D - getDM(D, \
                          np.random.normal(mag, 0.05), \
                          np.random.normal(gi, 0.05*1.4142), \
                          np.random.normal(Wlog, 0.005), \
                          np.random.normal(F21, 0.012), \
                          alfa[j], beta[j], \
                          slope[j], np.random.normal(zp[j], 0.033), \
                          V3k, \
                          np.random.normal(mc, 7.03018543e-07))

        D = D[~np.isnan(delta)]
        delta = delta[~np.isnan(delta)]

        try:
            D0 = interpol(D, delta)
            DM[i] = 5*np.log10(D0)+25
        except:
            D0 = np.max(D[delta<0])
            DM[i] = 5*np.log10(D0)+25
    except:
        pass
    

###########################################
t2 =  datetime.now()
print("Execution time:")
print(t2-t1)

2021-07-23 13:45:24.734375


<ipython-input-3-12b7b85dcfe7>:9: RuntimeWarning: invalid value encountered in log10
  Mag  = -2.5*(np.log10(Mb - 1.33*M21) - logX)


Execution time:
0:00:00.496073


In [5]:
np.std(DM)

0.12068038487017489

In [6]:
np.mean(DM)

35.35570134605284